<a href="https://colab.research.google.com/github/wjtopp3/CSIT-2033/blob/main/Week_13_Lab_1_SecureDesignReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Day 4 Exercise 1 Secure Design Review

In this group activity, you will take on the role of a security reviewer assessing a basic web application before deployment. The goal is to apply your knowledge of secure software design by identifying weaknesses in the system's architecture, assumptions, and implementation. Below you will find design notes, a test plan, and source code for a minimal Flask-based web service called SimpleReg, which performs basic user registration and message display.

While SimpleReg appears functional, it contains intentional security gaps that mirror common real-world oversights. As a team, conduct a structured Security Design Review, evaluating the application's functionality, data flows, and trust boundaries. This includes reviewing how user data is handled, identifying missing controls, and recommending mitigations using secure design principles. Be prepared to report findings and recommendations.

This is not just about finding bugs—it's about spotting architectural decisions that could be improved to reduce risk before code reaches production.

Use this simplified review checklist for your review (use the hardcopy if provided so you can include notes and comments).

---

## Security Design Review Checklist
### 1. Understand the System Context
- Are all application features (e.g., registration, welcome display) documented?
- Are trust boundaries (e.g., user input over HTTP) identified?
- Are the technologies and libraries (Flask, Python) clearly stated?

### 2. Authentication & Authorization
- Is there any authentication implemented for registration or viewing user data?
- Is access to `/welcome/<username>` restricted appropriately?
- Could one user impersonate or access another user’s data?

### 3. Input Validation & Output Handling
- Is user input validated or sanitized before storage?
- Is output encoded to prevent XSS in the welcome message?
- Can malicious input be used to manipulate the application's behavior?

### 4. Error Handling & Logging
- Are errors returned in a safe and user-friendly way?
- Could error messages leak system details (e.g., debug mode)?
- Is logging implemented or planned for future monitoring?

### 5. Secure Communications
- Is the app configured to run with HTTPS in production?

### 6. Dependency & Environment Management
- Are third-party libraries (e.g., Flask) kept up to date?
- Are there plans to manage secrets or configuration securely?

### 7. Review of Design Notes and Test Plan
- Do the design notes acknowledge key security assumptions and limitations?
- Does the test plan include basic security scenarios (e.g., XSS, input tampering)?
- Are there any missing test cases that would improve coverage (e.g., long input, malformed JSON)?


---

# **SimpleReg**
# Application Design Notes

## Purpose
A simple Flask web service for user registration and welcome message display.

## Functional Requirements
- Users submit username and email via a POST request to `/register`.
- The system stores this data in-memory (no database).
- Anyone can visit `/welcome/<username>` to see that user's welcome message.
- Output is returned as raw HTML.

## Technologies
- Python 3.11
- Flask 2.x

---

# Test Plan

## Manual Tests

### 1. Registration
- Submit valid JSON with username and email.
- Try submitting missing fields.
- Try registering the same username twice.

### 2. Welcome Page
- Access welcome page for a registered user.
- Access welcome page for an unregistered user.

### 3. Security Tests (Threat Simulation)
- Try XSS: Register with username = `<script>alert(1)</script>`
- Try injection in email field: `admin@example.com<script>`
- Manually test access to another user's info (no auth)

---

# Source Code

```
from flask import Flask, request, jsonify

app = Flask(__name__)
users = {}  # In-memory user store

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get("username")
    email = data.get("email")
    users[username] = email
    return jsonify({
      "message": f"User {username} registered with email {email}."})

@app.route('/welcome/<username>', methods=['GET'])
def welcome(username):
    email = users.get(username)
    if not email:
        return jsonify({"error": "User not found"}), 404

    return f"<h1>Welcome, {username}!</h1><p>Your email: {email}</p>"

if __name__ == '__main__':
    app.run(debug=True)
```


In [ ]:
!pip install flask
!pip install email_validator


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)
users = {}  # In-memory user store

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get("username")
    email = data.get("email")
    users[username] = email
    return jsonify({
      "message": f"User {username} registered with email {email}."})

@app.route('/welcome/<username>', methods=['GET'])
def welcome(username):
    email = users.get(username)
    if not email:
        return jsonify({"error": "User not found"}), 404

    return f"<h1>Welcome, {username}!</h1><p>Your email: {email}</p>"

if __name__ == '__main__':
    app.run(debug=True)